In [0]:
%sql
-- dim rider insertion
INSERT INTO dim_rider (Bike_Type)
SELECT rideable_type FROM silver_trip_data

In [0]:
%sql
-- dim station insertion
INSERT INTO dim_station (Station_Name, Latitude, Longitude)
SELECT DISTINCT 
    start_station_name, 
    start_lat, 
    start_lng
FROM silver_trip_data
WHERE start_station_name IS NOT NULL AND start_lat IS NOT NULL AND start_lng IS NOT NULL

UNION

SELECT DISTINCT 
    end_station_name, 
    end_lat, 
    end_lng
FROM silver_trip_data
WHERE end_station_name IS NOT NULL AND end_lat IS NOT NULL AND end_lng IS NOT NULL;

In [0]:
%sql
-- staging for dim weather (bin, text) 

CREATE OR REPLACE TEMPORARY VIEW staging_dim_weather AS
SELECT
    weather_ts, --natural key
    -- Temp Cat
    CASE
        WHEN temp_celsius <= 0 THEN 'Freezing'
        WHEN temp_celsius > 0 AND temp_celsius <= 10 THEN 'Cold'
        WHEN temp_celsius > 10 AND temp_celsius <= 20 THEN 'Cool'
        WHEN temp_celsius > 20 AND temp_celsius <= 30 THEN 'Warm'
        ELSE 'Hot'
    END AS Temp_type_text,
    
    CASE
        WHEN temp_celsius <= 0 THEN 10
        WHEN temp_celsius > 0 AND temp_celsius <= 10 THEN 20
        WHEN temp_celsius > 10 AND temp_celsius <= 20 THEN 30
        WHEN temp_celsius > 20 AND temp_celsius <= 30 THEN 40
        ELSE 50
    END AS Temp_type_bin, -- Bin code (e.g., 10, 20, 30...)
    
    -- Humidity Cat
    CASE
        WHEN hmdt_percent < 30 THEN 'Low'
        WHEN hmdt_percent >= 30 AND hmdt_percent <= 70 THEN 'Moderate'
        ELSE 'High'
    END AS Humidity_categories_text,
    
    CASE
        WHEN hmdt_percent < 30 THEN 1
        WHEN hmdt_percent >= 30 AND hmdt_percent <= 70 THEN 2
        ELSE 3
    END AS Humidity_categories_bin,
    
    -- Precipitation Cat
    CASE
        WHEN prcp_mm = 0 THEN 'None'
        WHEN prcp_mm > 0 AND prcp_mm <= 5 THEN 'Light Rain'
        WHEN prcp_mm > 5 AND prcp_mm <= 20 THEN 'Moderate Rain'
        ELSE 'Heavy Rain'
    END AS Precipitation_types_text,
    
    CASE
        WHEN prcp_mm = 0 THEN 1
        WHEN prcp_mm > 0 AND prcp_mm <= 5 THEN 2
        WHEN prcp_mm > 5 AND prcp_mm <= 20 THEN 3
        ELSE 4
    END AS Precipitation_types_bin,
    
    -- Wind Speed Cat
    CASE
        WHEN wnd_spd_kph <= 5 THEN 'Calm'
        WHEN wnd_spd_kph > 5 AND wnd_spd_kph <= 20 THEN 'Breezy'
        ELSE 'Windy'
    END AS Wind_type_categories_text,
    
    CASE
        WHEN wnd_spd_kph <= 5 THEN 1
        WHEN wnd_spd_kph > 5 AND wnd_spd_kph <= 20 THEN 2
        ELSE 3
    END AS Wind_type_categories_bin
    
FROM
    divvy.default.silver_weather_data
WHERE
    temp_celsius IS NOT NULL AND hmdt_percent IS NOT NULL;

In [0]:
%sql
-- dim weather insertion
INSERT INTO dim_weather (
  Weather_Key,
  Temp_type_text,
  Temp_type_bin,
  Humidity_categories_text,
  Humidity_categories_bin,
  Precipitation_types_text,
  Precipitation_types_bin,
  Wind_type_categories_text,
  Wind_type_categories_bin
)
SELECT
  weather_ts,
  Temp_type_text,
  Temp_type_bin,
  Humidity_categories_text,
  Humidity_categories_bin,
  Precipitation_types_text,
  Precipitation_types_bin,
  Wind_type_categories_text,
  Wind_type_categories_bin
FROM
staging_dim_weather;